In [ ]:
# Install dependencies
import sys
!{sys.executable} -m pip install 'itk>=5.0rc1' 'itkwidgets>=0.16.1'

In [2]:
import itk
import numpy as np
from itkwidgets import view

In [5]:
inputFileName = '1.nrrd'
image = itk.imread(inputFileName, itk.ctype('float'))
view(image,cmap="Grayscale", ui_collapsed=True, mode="x")

Viewer(cmap='Grayscale', gradient_opacity=0.22, mode='x', rendered_image=<itkImagePython.itkImageF3; proxy of …

In [6]:
# Note that source image includes the soft tissues of the cheeks and skin
view(image, cmap="Grayscale")

Viewer(cmap='Grayscale', gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageF3; proxy of <Swig Obje…

In [7]:
smoothed = itk.curvature_anisotropic_diffusion_image_filter(image,
                                                            time_step=0.0125,
                                                            number_of_iterations=5,
                                                            conductance_parameter=9.0)
view(smoothed, cmap="Grayscale", mode="x")

Viewer(cmap='Grayscale', gradient_opacity=0.22, mode='x', rendered_image=<itkImagePython.itkImageF3; proxy of …

In [9]:
# Compute edge image
gradient_magnitude = itk.gradient_magnitude_recursive_gaussian_image_filter(smoothed,
                                                                            sigma=0.4)
view(gradient_magnitude, ui_collapsed=True, cmap="Grayscale",mode="x")

Viewer(cmap='Grayscale', gradient_opacity=0.22, mode='x', rendered_image=<itkImagePython.itkImageF3; proxy of …

In [16]:
# Segment the bone only with a 3D region-growing algorithm
confidence_connected = itk.ConfidenceConnectedImageFilter.New(gradient_magnitude,
                                                              multiplier=2.5, 
                                                              number_of_iterations=7,
                                                              initial_neighborhood_radius=2,
                                                              replace_value=255)
confidence_connected.AddSeed([136, 61, 61])
confidence_connected.AddSeed([195, 132, 87])
confidence_connected.AddSeed([228, 163, 184])
confidence_connected.AddSeed([55, 154, 132])
confidence_connected.AddSeed([87, 78, 72])
confidence_connected.Update()

inverted = itk.InvertIntensityImageFilter(confidence_connected)
view(inverted, ui_collapsed=True, cmap="Grayscale")

Viewer(cmap='Grayscale', gradient_opacity=0.22, rendered_image=<itkImagePython.itkImageSS3; proxy of <Swig Obj…